タイタニックデータをいい感じに整理できた
- Nameを接頭語で分類。いくつかを統一
- チケットの接頭文字で階級を分類
- 客室番号で分類
- 家族の人数、一人かどうかのカラムを追加

あとできそうなことは
- 統一したNameごとに年齢の平均をとって、それを各欠損データに入れる
- FareとかAgeとか一定区切りごとに分解  
とか

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")
#train.head()

In [3]:
#trainの欠損を平均値、最頻値で埋める
train["Age"].fillna(train.Age.mean(), inplace=True)
train["Embarked"].fillna(0, inplace=True)

In [4]:
#Name列の整理
#正規表現で「(文字列).」の部分を抽出し、新たに作成する"Saluation"列に入れる　（Mr.とか入る）
#別表現の物を統一
#Mr,Miss,Mrs,Master,Rareにそれぞれ番号を振り、mappingする。満たさない奴はNanとなるのでそれを0で埋める
train['Salutation'] = train.Name.str.extract(' ([A-Za-z]+).', expand=False) 
train['Salutation'] = train['Salutation'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train['Salutation'] = train['Salutation'].replace('Mlle', 'Miss')
train['Salutation'] = train['Salutation'].replace('Ms', 'Miss')
train['Salutation'] = train['Salutation'].replace('Mme', 'Mrs')
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 
train['Salutation'] = train['Salutation'].map(Salutation_mapping) 
train['Salutation'] = train['Salutation'].fillna(0)
del train["Name"]
train.Salutation.unique()

array([1., 3., 2., 4., 0., 5.])

In [5]:
#Ticketの一番前の文字をTicket_Leftに格納
#apply:各要素に関数を適用
#無名関数lambda:要素xを入れて、一番前の文字を返す
train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
#文字型にしてるだけ
train['Ticket_Lett'] = train['Ticket_Lett'].apply(lambda x: str(x)) 
#指定の文字が先頭ならそのまま、ごく稀にある例外を0にする感じ
train['Ticket_Lett'] = np.where((train['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), train['Ticket_Lett'],0) 
#Ticket_Lenに長さを
train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x)) 
del train['Ticket']
#番号を降る。チケットの料金で合わせてるっぽい。
train['Ticket_Lett']=train['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3)

In [6]:
#客室番号で統一。A,C,Fで統一、B,D,Eで統一できるっぽい。
train['Cabin_Lett'] = train['Cabin'].apply(lambda x: str(x)[0]) 
train['Cabin_Lett'] = train['Cabin_Lett'].apply(lambda x: str(x)) 
train['Cabin_Lett'] = np.where((train['Cabin_Lett']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']),train['Cabin_Lett'], np.where((train['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0'))
del train['Cabin'] 
train['Cabin_Lett']=train['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)


In [7]:
#新しい変数「Family_size」導入
#Sibspは同乗する夫婦兄弟の数、Parchは同乗する親と子供の数より、
#Family_size = Sibsp + Parch + 1
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1

#新しい変数「IsAlone」追加
train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

In [8]:
#testも同じようにする。
test["Age"].fillna(train.Age.mean(), inplace=True)
test["Fare"].fillna(train.Fare.mean(), inplace=True)

test['Salutation'] = test.Name.str.extract(' ([A-Za-z]+).', expand=False) 
test['Salutation'] = test['Salutation'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test['Salutation'] = test['Salutation'].replace('Mlle', 'Miss')
test['Salutation'] = test['Salutation'].replace('Ms', 'Miss')
test['Salutation'] = test['Salutation'].replace('Mme', 'Mrs')
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 
test['Salutation'] = test['Salutation'].map(Salutation_mapping) 
test['Salutation'] = test['Salutation'].fillna(0)
del test["Name"]

test['Ticket_Lett'] = test['Ticket'].apply(lambda x: str(x)[0])
test['Ticket_Lett'] = test['Ticket_Lett'].apply(lambda x: str(x)) 
test['Ticket_Lett'] = np.where((test['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), test['Ticket_Lett'],0) 
test['Ticket_Len'] = test['Ticket'].apply(lambda x: len(x)) 
del test['Ticket']
test['Ticket_Lett']=test['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3)

test['Cabin_Lett'] = test['Cabin'].apply(lambda x: str(x)[0]) 
test['Cabin_Lett'] = test['Cabin_Lett'].apply(lambda x: str(x)) 
test['Cabin_Lett'] = np.where((test['Cabin_Lett']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']),test['Cabin_Lett'], np.where((test['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0'))
del test['Cabin'] 
test['Cabin_Lett']=test['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)

test["FamilySize"] = test["SibSp"] + test["Parch"] + 1
test['IsAlone'] = 0
test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1

In [9]:
train.head()
#test.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salutation,Ticket_Lett,Ticket_Len,Cabin_Lett,FamilySize,IsAlone
0,1,0,3,0,22.0,1,0,7.2500,0.0,1.0,3,9,0,2,0
1,2,1,1,1,38.0,1,0,71.2833,1.0,3.0,0,8,1,2,0
2,3,1,3,1,26.0,0,0,7.9250,0.0,2.0,3,16,0,1,1
3,4,1,1,1,35.0,1,0,53.1000,0.0,3.0,1,6,1,2,0
4,5,0,3,0,35.0,0,0,8.0500,0.0,1.0,3,6,0,1,1


In [10]:
train.to_csv("train_reshape.csv",index=False)
test.to_csv("test_reshape.csv",index=False)

In [11]:
train_data = train.values
X_train = train_data[:,2:]
y_train = train_data[:,1]

test_data = test.values
X_test = test_data[:,1:]

In [12]:
X_train

array([[ 3.        ,  0.        , 22.        , ...,  0.        ,
         2.        ,  0.        ],
       [ 1.        ,  1.        , 38.        , ...,  1.        ,
         2.        ,  0.        ],
       [ 3.        ,  1.        , 26.        , ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [ 3.        ,  1.        , 29.69911765, ...,  0.        ,
         4.        ,  0.        ],
       [ 1.        ,  0.        , 26.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 3.        ,  0.        , 32.        , ...,  0.        ,
         1.        ,  1.        ]])

In [13]:
from sklearn.ensemble import RandomForestClassifier
random_forest=RandomForestClassifier()
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test)
Y_pred = Y_pred.astype(int)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [16]:
pid = np.array(test["PassengerId"])

In [17]:
sol = pd.DataFrame({"PassengerId":pid,"Survived":Y_pred})

In [18]:
sol.to_csv("sol.csv",index=False)